# 14 - SIFT Two-View Pose (Notebook 12 + Corner-Guided Ideas)

Dieses Notebook kombiniert die robuste Daten-/Pair-Logik aus Notebook 12 mit den Ideen aus Notebook 14:
- optionales Corner-Guiding fuer SIFT (`goodFeaturesToTrack` + `SIFT.compute`)
- RootSIFT
- Fokusmasken gegen Gras-Textur (`none`, `nongreen`, `edge_corner_mild`, `edge_corner_strict`, `line_corner_only`)
- Homography-guided Essential/Pose-Auswertung

Ziel: robuste relative Pose (Rotation/Translation/Scale in Pixeln) fuer ein aufeinanderfolgendes Bildpaar.


In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

plt.rcParams['figure.dpi'] = 130
np.set_printoptions(precision=4, suppress=True)


In [ ]:
# Basic checks + paths + global config
if not hasattr(cv2, 'SIFT_create'):
    raise RuntimeError('OpenCV build has no SIFT. Please install opencv-contrib-python.')

CANDIDATE_ROOTS = [Path.cwd(), Path.cwd().parent, Path('.'), Path('..')]
PROJECT_ROOT = None
_seen = set()
for cand in CANDIDATE_ROOTS:
    try:
        root = cand.resolve()
    except Exception:
        continue
    k = str(root)
    if k in _seen:
        continue
    _seen.add(k)
    if (root / 'data' / 'data').exists():
        PROJECT_ROOT = root
        break
if PROJECT_ROOT is None:
    raise FileNotFoundError('Could not find project root containing data/data.')

DATA_ROOT = PROJECT_ROOT / 'data' / 'data'
TRAIN_IMG_DIR = DATA_ROOT / 'train_data' / 'train_images'
TRAIN_POS_CSV = DATA_ROOT / 'train_data' / 'train_pos.csv'
TRAIN_CAM_CSV = DATA_ROOT / 'train_data' / 'train_cam.csv'

# Pair selection
PAIR_ANCHOR_ID = 13
PAIR_VAL_ID = 14
PREVIEW_PAIR_INDEX = 0

# Runtime + preprocessing
IMAGE_MAX_SIDE = 1400  # None for full resolution
PREPROCESS_MODE = 'gray_denoise_clahe'  # ['gray', 'gray_clahe', 'gray_denoise_clahe']

# Geometry thresholds
AFFINE_RANSAC_THR = 3.0
HOMOGRAPHY_RANSAC_THR = 4.0
ESSENTIAL_RANSAC_THR = 1.5  # in normalized coordinates (after undistortPoints)

# Match filtering
H_GUIDE_ERR_PX = 4.0
MIN_MATCHES_FOR_GEOM = 8

# Draw control
MAX_DRAW_MATCHES = 500

print('project_root:', PROJECT_ROOT)
print('train_img_dir:', TRAIN_IMG_DIR)


In [ ]:
# Load train table + consecutive pairs
train_pos_df = pd.read_csv(TRAIN_POS_CSV)
train_cam_df = pd.read_csv(TRAIN_CAM_CSV)
train_df = train_cam_df.merge(train_pos_df, on='id', how='inner').copy()
train_df['id'] = train_df['id'].astype(int)
train_df = train_df.sort_values('id').reset_index(drop=True)

required_cols = ['id', 'x_pixel', 'y_pixel', 'fx', 'fy', 'cx', 'cy']
for c in required_cols:
    if c not in train_df.columns:
        raise KeyError(f'Missing required column: {c}')

pairs = []
for i in range(len(train_df) - 1):
    r0 = train_df.iloc[i]
    r1 = train_df.iloc[i + 1]
    id0 = int(r0['id'])
    id1 = int(r1['id'])
    if id1 != id0 + 1:
        continue
    pairs.append({'pair_idx': len(pairs), 'anchor_id': id0, 'val_id': id1})
pair_df = pd.DataFrame(pairs)

print('consecutive pairs:', len(pair_df))
display(pair_df.head(20))


In [ ]:
# Utilities: image loading, preprocess, intrinsics
_IMAGE_CACHE: Dict[Tuple[int, Optional[int]], Tuple[np.ndarray, float]] = {}


def resolve_train_image_path(image_id: int) -> Path:
    stems = [f'{int(image_id):04d}', str(int(image_id))]
    exts = ['.JPG', '.jpg', '.jpeg', '.JPEG', '.png', '.PNG']
    for st in stems:
        for ext in exts:
            p = TRAIN_IMG_DIR / f'{st}{ext}'
            if p.exists():
                return p
    raise FileNotFoundError(f'Image not found for id={image_id} in {TRAIN_IMG_DIR}')


def load_train_image_cached(image_id: int, max_side: Optional[int]) -> Tuple[np.ndarray, float]:
    key = (int(image_id), max_side)
    if key in _IMAGE_CACHE:
        return _IMAGE_CACHE[key]

    p = resolve_train_image_path(int(image_id))
    bgr = cv2.imread(str(p), cv2.IMREAD_COLOR)
    if bgr is None:
        raise RuntimeError(f'Cannot read image: {p}')
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    scale = 1.0
    if max_side is not None:
        h, w = rgb.shape[:2]
        m = max(h, w)
        if m > int(max_side):
            scale = float(max_side) / float(m)
            nw = max(32, int(round(w * scale)))
            nh = max(32, int(round(h * scale)))
            rgb = cv2.resize(rgb, (nw, nh), interpolation=cv2.INTER_AREA)

    _IMAGE_CACHE[key] = (rgb, scale)
    return rgb, scale


def preprocess_for_sift(img_rgb: np.ndarray, mode: str = 'gray_clahe') -> np.ndarray:
    g = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    if mode == 'gray':
        return g
    if mode == 'gray_clahe':
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        return clahe.apply(g)
    if mode == 'gray_denoise_clahe':
        g2 = cv2.bilateralFilter(g, d=7, sigmaColor=45, sigmaSpace=45)
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
        return clahe.apply(g2)
    raise KeyError(f'Unknown preprocess mode: {mode}')


def scaled_K(cam_row: pd.Series, image_scale: float) -> np.ndarray:
    fx = float(cam_row['fx']) * float(image_scale)
    fy = float(cam_row['fy']) * float(image_scale)
    cx = float(cam_row['cx']) * float(image_scale)
    cy = float(cam_row['cy']) * float(image_scale)
    return np.array([[fx, 0.0, cx], [0.0, fy, cy], [0.0, 0.0, 1.0]], dtype=np.float64)


def select_pair_row(pair_df: pd.DataFrame, anchor_id: int, val_id: int, fallback_idx: int = 0) -> pd.Series:
    sel = pair_df[
        (pair_df['anchor_id'].astype(int) == int(anchor_id)) &
        (pair_df['val_id'].astype(int) == int(val_id))
    ]
    if len(sel) > 0:
        return sel.iloc[0]
    idx = int(np.clip(fallback_idx, 0, len(pair_df) - 1))
    print(f'Pair {anchor_id}->{val_id} not found, fallback index {idx}.')
    return pair_df.iloc[idx]


In [ ]:
# Feature-focus masks (less grass, more structure)
def _ensure_gray_u8(gray: np.ndarray) -> np.ndarray:
    if gray.dtype == np.uint8:
        return gray
    return np.clip(gray, 0, 255).astype(np.uint8)


def compute_green_mask(
    img_rgb: np.ndarray,
    h_low: int = 30,
    h_high: int = 100,
    s_min: int = 35,
    v_min: int = 25,
) -> np.ndarray:
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    lo = np.array([int(h_low), int(s_min), int(v_min)], dtype=np.uint8)
    hi = np.array([int(h_high), 255, 255], dtype=np.uint8)
    return cv2.inRange(hsv, lo, hi)


def _draw_hough_line_mask(edges: np.ndarray, threshold: int, min_line_len: int, max_line_gap: int, thickness: int) -> np.ndarray:
    line_mask = np.zeros_like(edges, dtype=np.uint8)
    lines = cv2.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi / 180.0,
        threshold=int(threshold),
        minLineLength=int(min_line_len),
        maxLineGap=int(max_line_gap),
    )
    if lines is not None:
        for ln in lines:
            x1, y1, x2, y2 = ln[0]
            cv2.line(line_mask, (int(x1), int(y1)), (int(x2), int(y2)), 255, int(thickness), cv2.LINE_AA)
    return line_mask


def _draw_corner_mask(gray_u8: np.ndarray, max_corners: int, quality: float, min_dist: float, radius: int) -> np.ndarray:
    corner_mask = np.zeros_like(gray_u8, dtype=np.uint8)
    corners = cv2.goodFeaturesToTrack(
        gray_u8,
        maxCorners=int(max_corners),
        qualityLevel=float(quality),
        minDistance=float(min_dist),
        useHarrisDetector=False,
    )
    if corners is not None:
        for c in corners.reshape(-1, 2):
            x = int(round(float(c[0])))
            y = int(round(float(c[1])))
            cv2.circle(corner_mask, (x, y), int(radius), 255, -1, cv2.LINE_AA)
    return corner_mask


def build_feature_focus_mask(
    img_rgb: np.ndarray,
    gray: np.ndarray,
    profile: str,
    params: Optional[Dict[str, float]] = None,
) -> Tuple[np.ndarray, Dict[str, float], np.ndarray]:
    p = dict(params or {})
    gray_u8 = _ensure_gray_u8(gray)

    full = np.full_like(gray_u8, 255, dtype=np.uint8)
    green = compute_green_mask(
        img_rgb,
        h_low=int(p.get('green_h_low', 30)),
        h_high=int(p.get('green_h_high', 100)),
        s_min=int(p.get('green_s_min', 35)),
        v_min=int(p.get('green_v_min', 25)),
    )
    nongreen = cv2.bitwise_not(green)

    k_open = int(max(1, p.get('nongreen_open', 3)))
    k_close = int(max(1, p.get('nongreen_close', 5)))
    nongreen = cv2.morphologyEx(nongreen, cv2.MORPH_OPEN, np.ones((k_open, k_open), np.uint8))
    nongreen = cv2.morphologyEx(nongreen, cv2.MORPH_CLOSE, np.ones((k_close, k_close), np.uint8))

    canny_low = int(p.get('canny_low', 70))
    canny_high = int(p.get('canny_high', 170))
    edges = cv2.Canny(gray_u8, canny_low, canny_high)

    edge_dilate = int(max(1, p.get('edge_dilate', 2)))
    if edge_dilate > 1:
        edges = cv2.dilate(edges, np.ones((edge_dilate, edge_dilate), np.uint8), iterations=1)

    line_mask = _draw_hough_line_mask(
        edges=edges,
        threshold=int(p.get('line_threshold', 65)),
        min_line_len=int(p.get('line_min_len', 60)),
        max_line_gap=int(p.get('line_max_gap', 8)),
        thickness=int(p.get('line_thickness', 2)),
    )
    corner_mask = _draw_corner_mask(
        gray_u8=gray_u8,
        max_corners=int(p.get('corner_max', 1200)),
        quality=float(p.get('corner_quality', 0.010)),
        min_dist=float(p.get('corner_min_dist', 6.0)),
        radius=int(p.get('corner_radius', 2)),
    )

    line_dilate = int(max(1, p.get('line_dilate', 2)))
    corner_dilate = int(max(1, p.get('corner_dilate', 2)))
    if line_dilate > 1:
        line_mask = cv2.dilate(line_mask, np.ones((line_dilate, line_dilate), np.uint8), iterations=1)
    if corner_dilate > 1:
        corner_mask = cv2.dilate(corner_mask, np.ones((corner_dilate, corner_dilate), np.uint8), iterations=1)

    structure = cv2.bitwise_or(edges, line_mask)
    structure = cv2.bitwise_or(structure, corner_mask)

    profile = str(profile).strip().lower()
    if profile == 'none':
        mask = full
    elif profile == 'nongreen':
        mask = nongreen
    elif profile == 'edge_corner_mild':
        mask = cv2.bitwise_and(nongreen, cv2.dilate(structure, np.ones((3, 3), np.uint8), iterations=1))
        mask = cv2.bitwise_or(mask, line_mask)
        mask = cv2.bitwise_or(mask, corner_mask)
    elif profile == 'edge_corner_strict':
        mask = cv2.bitwise_and(nongreen, structure)
        mask = cv2.bitwise_or(mask, cv2.bitwise_and(line_mask, nongreen))
        mask = cv2.bitwise_or(mask, cv2.bitwise_and(corner_mask, nongreen))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    elif profile == 'line_corner_only':
        mask = cv2.bitwise_or(line_mask, corner_mask)
    else:
        raise KeyError(f'Unknown mask profile: {profile}')

    final_close = int(max(1, p.get('final_close', 3)))
    if final_close > 1:
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((final_close, final_close), np.uint8))

    min_cov = float(p.get('min_coverage', 0.03))
    cov = float((mask > 0).mean())
    fallback = 0.0
    if cov < min_cov:
        mask = nongreen.copy()
        cov = float((mask > 0).mean())
        fallback = 1.0

    diag = {
        'mask_cov': cov,
        'green_cov': float((green > 0).mean()),
        'edge_cov': float((edges > 0).mean()),
        'line_cov': float((line_mask > 0).mean()),
        'corner_cov': float((corner_mask > 0).mean()),
        'fallback': fallback,
    }
    return mask.astype(np.uint8), diag, green.astype(np.uint8)


def _mask_tint(gray: np.ndarray, mask: np.ndarray) -> np.ndarray:
    base = cv2.cvtColor(_ensure_gray_u8(gray), cv2.COLOR_GRAY2RGB)
    vis = base.copy()
    blocked = mask <= 0
    if np.any(blocked):
        red = np.array([255, 45, 45], dtype=np.uint8)
        vis[blocked] = (0.35 * vis[blocked] + 0.65 * red).astype(np.uint8)
    return vis


In [ ]:
# SIFT extraction + matching (RootSIFT, corner-guided, grid-select)
def rootsift(desc: Optional[np.ndarray], eps: float = 1e-12) -> Optional[np.ndarray]:
    if desc is None:
        return None
    desc = desc.astype(np.float32)
    desc /= (np.sum(np.abs(desc), axis=1, keepdims=True) + eps)
    desc = np.sqrt(np.clip(desc, 0.0, None))
    return desc


def grid_select_keypoints(
    kps: List[cv2.KeyPoint],
    desc: np.ndarray,
    img_shape: Tuple[int, int],
    grid_shape: Tuple[int, int] = (8, 8),
    per_cell: int = 200,
) -> Tuple[List[cv2.KeyPoint], np.ndarray]:
    if len(kps) == 0 or desc is None or len(desc) == 0:
        return [], np.zeros((0, 128), dtype=np.float32)

    h, w = img_shape[:2]
    gy, gx = int(grid_shape[0]), int(grid_shape[1])
    cell_w, cell_h = max(1.0, w / gx), max(1.0, h / gy)

    buckets = [[[] for _ in range(gx)] for _ in range(gy)]
    for i, kp in enumerate(kps):
        x, y = kp.pt
        cx = min(gx - 1, int(x / cell_w))
        cy = min(gy - 1, int(y / cell_h))
        buckets[cy][cx].append((float(kp.response), i))

    keep_idx: List[int] = []
    for cy in range(gy):
        for cx in range(gx):
            buckets[cy][cx].sort(key=lambda t: t[0], reverse=True)
            keep_idx.extend([i for _, i in buckets[cy][cx][:int(per_cell)]])

    if len(keep_idx) == 0:
        return [], np.zeros((0, desc.shape[1]), dtype=desc.dtype)

    keep_idx_arr = np.array(keep_idx, dtype=np.int32)
    kps_sel = [kps[int(i)] for i in keep_idx_arr]
    desc_sel = desc[keep_idx_arr]
    return kps_sel, desc_sel


def make_corner_keypoints(
    gray: np.ndarray,
    mask: Optional[np.ndarray],
    max_corners: int,
    quality_level: float,
    min_distance: float,
    block_size: int,
    use_harris: bool,
    kp_size: float,
) -> List[cv2.KeyPoint]:
    corners = cv2.goodFeaturesToTrack(
        gray,
        maxCorners=int(max_corners),
        qualityLevel=float(quality_level),
        minDistance=float(min_distance),
        mask=mask,
        blockSize=int(block_size),
        useHarrisDetector=bool(use_harris),
    )
    if corners is None:
        return []
    out: List[cv2.KeyPoint] = []
    for c in corners.reshape(-1, 2):
        out.append(cv2.KeyPoint(float(c[0]), float(c[1]), float(kp_size)))
    return out


def extract_sift_features(gray: np.ndarray, mask: Optional[np.ndarray], cfg: Dict[str, object]) -> Tuple[List[cv2.KeyPoint], Optional[np.ndarray]]:
    sift = cv2.SIFT_create(
        nfeatures=int(cfg.get('nfeatures', 9000)),
        contrastThreshold=float(cfg.get('contrast_thr', 0.03)),
        edgeThreshold=float(cfg.get('edge_thr', 12.0)),
        sigma=float(cfg.get('sigma', 1.6)),
    )

    use_corner_guided = bool(cfg.get('use_corner_guided', False))
    if use_corner_guided:
        guided_kps = make_corner_keypoints(
            gray=gray,
            mask=mask,
            max_corners=int(cfg.get('corner_max', 6000)),
            quality_level=float(cfg.get('corner_quality', 0.01)),
            min_distance=float(cfg.get('corner_min_dist', 6.0)),
            block_size=int(cfg.get('corner_block_size', 7)),
            use_harris=bool(cfg.get('corner_use_harris', False)),
            kp_size=float(cfg.get('corner_kp_size', 16.0)),
        )
        if len(guided_kps) == 0:
            return [], None
        kps, desc = sift.compute(gray, guided_kps)
    else:
        kps, desc = sift.detectAndCompute(gray, mask)

    if desc is None or kps is None or len(kps) == 0:
        return [], None

    if bool(cfg.get('use_rootsift', True)):
        desc = rootsift(desc)

    if bool(cfg.get('use_grid_selection', False)):
        kps, desc = grid_select_keypoints(
            kps=kps,
            desc=desc,
            img_shape=gray.shape,
            grid_shape=tuple(cfg.get('grid_shape', (8, 8))),
            per_cell=int(cfg.get('grid_per_cell', 180)),
        )

    return kps, desc


def knn_ratio_matches(desc0: np.ndarray, desc1: np.ndarray, ratio: float = 0.75) -> List[cv2.DMatch]:
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    knn = bf.knnMatch(desc0, desc1, k=2)
    good: List[cv2.DMatch] = []
    for pair in knn:
        if len(pair) < 2:
            continue
        m, n = pair
        if m.distance < float(ratio) * n.distance:
            good.append(m)
    return good


def mutual_filter(matches_01: List[cv2.DMatch], matches_10: List[cv2.DMatch]) -> List[cv2.DMatch]:
    map_01 = {(int(m.queryIdx), int(m.trainIdx)): m for m in matches_01}
    mutual: List[cv2.DMatch] = []
    for m in matches_10:
        key = (int(m.trainIdx), int(m.queryIdx))
        if key in map_01:
            mutual.append(map_01[key])
    return mutual


def match_descriptors(desc0: Optional[np.ndarray], desc1: Optional[np.ndarray], ratio: float, mutual: bool) -> List[cv2.DMatch]:
    if desc0 is None or desc1 is None or len(desc0) < 2 or len(desc1) < 2:
        return []
    m01 = knn_ratio_matches(desc0, desc1, ratio=float(ratio))
    if not bool(mutual):
        return m01
    m10 = knn_ratio_matches(desc1, desc0, ratio=float(ratio))
    return mutual_filter(m01, m10)


def matches_to_points(kps0: List[cv2.KeyPoint], kps1: List[cv2.KeyPoint], matches: List[cv2.DMatch]) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    if len(matches) == 0:
        return (
            np.zeros((0, 2), dtype=np.float32),
            np.zeros((0, 2), dtype=np.float32),
            np.zeros((0,), dtype=np.float32),
            np.zeros((0,), dtype=np.float32),
        )
    pts0 = np.array([kps0[m.queryIdx].pt for m in matches], dtype=np.float32)
    pts1 = np.array([kps1[m.trainIdx].pt for m in matches], dtype=np.float32)
    s0 = np.array([kps0[m.queryIdx].size for m in matches], dtype=np.float32)
    s1 = np.array([kps1[m.trainIdx].size for m in matches], dtype=np.float32)
    return pts0, pts1, s0, s1


In [ ]:
# Geometry + result structure
def rotation_matrix_to_euler_zyx(R: np.ndarray) -> Tuple[float, float, float]:
    sy = float(np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2))
    singular = sy < 1e-6
    if not singular:
        yaw = np.arctan2(R[1, 0], R[0, 0])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[2, 1], R[2, 2])
    else:
        yaw = np.arctan2(-R[0, 1], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0.0
    return float(np.degrees(yaw)), float(np.degrees(pitch)), float(np.degrees(roll))


def affine_transform_points(M: np.ndarray, pts: np.ndarray) -> np.ndarray:
    if pts.shape[0] == 0:
        return pts.copy()
    ones = np.ones((pts.shape[0], 1), dtype=np.float32)
    p = np.hstack([pts.astype(np.float32), ones])
    q = (M @ p.T).T
    return q.astype(np.float32)


def estimate_affine_partial(pts0: np.ndarray, pts1: np.ndarray, thr_px: float) -> Tuple[Optional[np.ndarray], np.ndarray]:
    if pts0.shape[0] < 3:
        return None, np.zeros((pts0.shape[0],), dtype=bool)
    M, inl = cv2.estimateAffinePartial2D(
        pts1,
        pts0,
        method=cv2.RANSAC,
        ransacReprojThreshold=float(thr_px),
        maxIters=20000,
        confidence=0.999,
        refineIters=50,
    )
    if inl is None:
        return M, np.zeros((pts0.shape[0],), dtype=bool)
    return M, inl.ravel().astype(bool)


def estimate_homography(pts0: np.ndarray, pts1: np.ndarray, thr_px: float) -> Tuple[Optional[np.ndarray], np.ndarray]:
    if pts0.shape[0] < 4:
        return None, np.zeros((pts0.shape[0],), dtype=bool)
    method = cv2.USAC_MAGSAC if hasattr(cv2, 'USAC_MAGSAC') else cv2.RANSAC
    H, inl = cv2.findHomography(
        pts1,
        pts0,
        method=method,
        ransacReprojThreshold=float(thr_px),
        maxIters=20000,
        confidence=0.999,
    )
    if inl is None:
        return H, np.zeros((pts0.shape[0],), dtype=bool)
    return H, inl.ravel().astype(bool)


def homography_transfer_error(H: Optional[np.ndarray], pts1: np.ndarray, pts0: np.ndarray) -> np.ndarray:
    if H is None or pts0.shape[0] == 0:
        return np.full((pts0.shape[0],), np.inf, dtype=np.float32)
    pts1_h = cv2.convertPointsToHomogeneous(pts1).reshape(-1, 3)
    proj = (H @ pts1_h.T).T
    denom = np.clip(proj[:, 2:3], 1e-8, None)
    proj_xy = proj[:, :2] / denom
    err = np.linalg.norm(proj_xy - pts0, axis=1)
    return err.astype(np.float32)


def guided_filter_by_H(H: Optional[np.ndarray], pts0: np.ndarray, pts1: np.ndarray, max_err_px: float) -> np.ndarray:
    if pts0.shape[0] == 0:
        return np.zeros((0,), dtype=bool)
    if H is None:
        return np.ones((pts0.shape[0],), dtype=bool)
    err = homography_transfer_error(H, pts1, pts0)
    return err < float(max_err_px)


def estimate_essential_pose(
    pts0: np.ndarray,
    pts1: np.ndarray,
    K0: np.ndarray,
    K1: np.ndarray,
    thr_norm: float,
) -> Tuple[Optional[np.ndarray], np.ndarray, Optional[np.ndarray], Optional[np.ndarray], np.ndarray]:
    if pts0.shape[0] < 8:
        n = pts0.shape[0]
        return None, np.zeros((n,), dtype=bool), None, None, np.zeros((n,), dtype=bool)

    pts0n = cv2.undistortPoints(pts0.reshape(-1, 1, 2), K0, None).reshape(-1, 2)
    pts1n = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K1, None).reshape(-1, 2)

    E, inl = cv2.findEssentialMat(
        pts0n,
        pts1n,
        cameraMatrix=np.eye(3),
        method=cv2.RANSAC,
        prob=0.999,
        threshold=float(thr_norm),
    )

    if inl is None:
        n = pts0.shape[0]
        return E, np.zeros((n,), dtype=bool), None, None, np.zeros((n,), dtype=bool)

    inl = inl.ravel().astype(bool)
    if np.count_nonzero(inl) < 8 or E is None:
        return E, inl, None, None, np.zeros_like(inl)

    _, R, t, inl_pose = cv2.recoverPose(
        E,
        pts0n,
        pts1n,
        cameraMatrix=np.eye(3),
        mask=inl.astype(np.uint8).reshape(-1, 1),
    )
    if inl_pose is None:
        inl_pose = inl.copy().astype(np.uint8).reshape(-1, 1)
    return E, inl, R, t.reshape(3), inl_pose.ravel().astype(bool)


def _pts_in_mask(mask_u8: np.ndarray, pts: np.ndarray) -> np.ndarray:
    if pts.shape[0] == 0:
        return np.zeros((0,), dtype=bool)
    h, w = mask_u8.shape[:2]
    out = np.zeros((pts.shape[0],), dtype=bool)
    for i, p in enumerate(pts):
        x = int(np.clip(round(float(p[0])), 0, w - 1))
        y = int(np.clip(round(float(p[1])), 0, h - 1))
        out[i] = bool(mask_u8[y, x] > 0)
    return out


def _kp_green_fraction(kps: List[cv2.KeyPoint], green_mask_u8: np.ndarray) -> float:
    if len(kps) == 0:
        return np.nan
    h, w = green_mask_u8.shape[:2]
    c = 0
    for kp in kps:
        x = int(np.clip(round(float(kp.pt[0])), 0, w - 1))
        y = int(np.clip(round(float(kp.pt[1])), 0, h - 1))
        if green_mask_u8[y, x] > 0:
            c += 1
    return 100.0 * float(c) / float(len(kps))


@dataclass
class VariantResult:
    name: str
    preprocess_mode: str
    mask_profile: str
    gray0: np.ndarray
    gray1: np.ndarray
    mask0: np.ndarray
    mask1: np.ndarray
    green0: np.ndarray
    green1: np.ndarray
    mask_diag0: Dict[str, float]
    mask_diag1: Dict[str, float]
    keypoints0: List[cv2.KeyPoint]
    keypoints1: List[cv2.KeyPoint]
    matches: List[cv2.DMatch]
    pts0: np.ndarray
    pts1: np.ndarray
    kp_size0: np.ndarray
    kp_size1: np.ndarray
    affine_M_1to0: Optional[np.ndarray]
    affine_inliers: np.ndarray
    affine_rotation_deg: float
    affine_tx: float
    affine_ty: float
    affine_scale: float
    affine_rmse: float
    H_1to0: Optional[np.ndarray]
    homography_inliers: np.ndarray
    guided_mask: np.ndarray
    sfm_inliers: np.ndarray
    sfm_R: Optional[np.ndarray]
    sfm_t: Optional[np.ndarray]
    sfm_rot_deg: float
    sfm_yaw_deg: float
    sfm_pitch_deg: float
    sfm_roll_deg: float
    essential_inliers_raw: np.ndarray
    dropped_by_response: int
    dropped_by_size_ratio: int


In [ ]:
# Full variant runner
def _empty_variant_result(
    name: str,
    preprocess_mode: str,
    mask_profile: str,
    gray0: np.ndarray,
    gray1: np.ndarray,
    mask0: np.ndarray,
    mask1: np.ndarray,
    green0: np.ndarray,
    green1: np.ndarray,
    diag0: Dict[str, float],
    diag1: Dict[str, float],
    keypoints0: Optional[List[cv2.KeyPoint]] = None,
    keypoints1: Optional[List[cv2.KeyPoint]] = None,
) -> VariantResult:
    return VariantResult(
        name=name,
        preprocess_mode=preprocess_mode,
        mask_profile=mask_profile,
        gray0=gray0,
        gray1=gray1,
        mask0=mask0,
        mask1=mask1,
        green0=green0,
        green1=green1,
        mask_diag0=diag0,
        mask_diag1=diag1,
        keypoints0=keypoints0 if keypoints0 is not None else [],
        keypoints1=keypoints1 if keypoints1 is not None else [],
        matches=[],
        pts0=np.zeros((0, 2), dtype=np.float32),
        pts1=np.zeros((0, 2), dtype=np.float32),
        kp_size0=np.zeros((0,), dtype=np.float32),
        kp_size1=np.zeros((0,), dtype=np.float32),
        affine_M_1to0=None,
        affine_inliers=np.zeros((0,), dtype=bool),
        affine_rotation_deg=np.nan,
        affine_tx=np.nan,
        affine_ty=np.nan,
        affine_scale=np.nan,
        affine_rmse=np.nan,
        H_1to0=None,
        homography_inliers=np.zeros((0,), dtype=bool),
        guided_mask=np.zeros((0,), dtype=bool),
        sfm_inliers=np.zeros((0,), dtype=bool),
        sfm_R=None,
        sfm_t=None,
        sfm_rot_deg=np.nan,
        sfm_yaw_deg=np.nan,
        sfm_pitch_deg=np.nan,
        sfm_roll_deg=np.nan,
        essential_inliers_raw=np.zeros((0,), dtype=bool),
        dropped_by_response=0,
        dropped_by_size_ratio=0,
    )


def run_sift_corner_variant(
    img0_rgb: np.ndarray,
    img1_rgb: np.ndarray,
    K0: np.ndarray,
    K1: np.ndarray,
    cfg: Dict[str, object],
) -> VariantResult:
    name = str(cfg['name'])
    preprocess_mode = str(cfg.get('preprocess_mode', PREPROCESS_MODE))
    mask_profile = str(cfg.get('mask_profile', 'none'))

    gray0 = preprocess_for_sift(img0_rgb, preprocess_mode)
    gray1 = preprocess_for_sift(img1_rgb, preprocess_mode)

    mask0, diag0, green0 = build_feature_focus_mask(img0_rgb, gray0, mask_profile, cfg.get('mask_params', {}))
    mask1, diag1, green1 = build_feature_focus_mask(img1_rgb, gray1, mask_profile, cfg.get('mask_params', {}))

    k0, d0 = extract_sift_features(gray0, mask0, cfg)
    k1, d1 = extract_sift_features(gray1, mask1, cfg)

    if d0 is None or d1 is None or len(k0) == 0 or len(k1) == 0:
        return _empty_variant_result(name, preprocess_mode, mask_profile, gray0, gray1, mask0, mask1, green0, green1, diag0, diag1, k0, k1)

    ratio = float(cfg.get('ratio_thr', 0.78))
    mutual = bool(cfg.get('mutual_check', False))
    matches = match_descriptors(d0, d1, ratio=ratio, mutual=mutual)

    if len(matches) == 0:
        return _empty_variant_result(name, preprocess_mode, mask_profile, gray0, gray1, mask0, mask1, green0, green1, diag0, diag1, k0, k1)

    pts0, pts1, s0, s1 = matches_to_points(k0, k1, matches)

    kp_response_min = float(cfg.get('kp_response_min', 0.0))
    size_ratio_min = float(cfg.get('size_ratio_min', 0.0))
    size_ratio_max = float(cfg.get('size_ratio_max', 99.0))

    keep = np.ones((pts0.shape[0],), dtype=bool)
    dropped_resp = 0
    dropped_size = 0
    for i, m in enumerate(matches):
        if not keep[i]:
            continue
        r0 = float(k0[m.queryIdx].response)
        r1 = float(k1[m.trainIdx].response)
        if min(r0, r1) < kp_response_min:
            keep[i] = False
            dropped_resp += 1
            continue
        sr = (float(k1[m.trainIdx].size) + 1e-6) / (float(k0[m.queryIdx].size) + 1e-6)
        if sr < size_ratio_min or sr > size_ratio_max:
            keep[i] = False
            dropped_size += 1

    if not np.any(keep):
        out = _empty_variant_result(name, preprocess_mode, mask_profile, gray0, gray1, mask0, mask1, green0, green1, diag0, diag1, k0, k1)
        out.dropped_by_response = dropped_resp
        out.dropped_by_size_ratio = dropped_size
        return out

    matches = [m for i, m in enumerate(matches) if keep[i]]
    pts0 = pts0[keep]
    pts1 = pts1[keep]
    s0 = s0[keep]
    s1 = s1[keep]

    M, inl_aff = estimate_affine_partial(pts0, pts1, thr_px=float(cfg.get('affine_thr', AFFINE_RANSAC_THR)))
    H, inl_h = estimate_homography(pts0, pts1, thr_px=float(cfg.get('homography_thr', HOMOGRAPHY_RANSAC_THR)))

    if M is not None and inl_aff.size > 0:
        rot_deg = float(np.degrees(np.arctan2(M[1, 0], M[0, 0])))
        scale = float(np.sqrt(M[0, 0] ** 2 + M[1, 0] ** 2))
        tx = float(M[0, 2])
        ty = float(M[1, 2])

        p1w = affine_transform_points(M, pts1)
        if np.any(inl_aff):
            err = np.linalg.norm(pts0[inl_aff] - p1w[inl_aff], axis=1)
            rmse = float(np.sqrt(np.mean(err ** 2)))
        else:
            rmse = np.nan
    else:
        rot_deg = np.nan
        scale = np.nan
        tx = np.nan
        ty = np.nan
        rmse = np.nan

    guided_mask = guided_filter_by_H(H, pts0, pts1, max_err_px=float(cfg.get('h_guide_err_px', H_GUIDE_ERR_PX)))
    pts0_g = pts0[guided_mask]
    pts1_g = pts1[guided_mask]

    _, inl_e_raw, _, _, _ = estimate_essential_pose(
        pts0=pts0,
        pts1=pts1,
        K0=K0,
        K1=K1,
        thr_norm=float(cfg.get('essential_thr', ESSENTIAL_RANSAC_THR)),
    )

    _, inl_e, R, t, inl_pose = estimate_essential_pose(
        pts0=pts0_g,
        pts1=pts1_g,
        K0=K0,
        K1=K1,
        thr_norm=float(cfg.get('essential_thr', ESSENTIAL_RANSAC_THR)),
    )

    sfm_rot = np.nan
    sfm_yaw = np.nan
    sfm_pitch = np.nan
    sfm_roll = np.nan
    if R is not None:
        rvec, _ = cv2.Rodrigues(R)
        sfm_rot = float(np.linalg.norm(rvec) * 180.0 / np.pi)
        sfm_yaw, sfm_pitch, sfm_roll = rotation_matrix_to_euler_zyx(R)

    return VariantResult(
        name=name,
        preprocess_mode=preprocess_mode,
        mask_profile=mask_profile,
        gray0=gray0,
        gray1=gray1,
        mask0=mask0,
        mask1=mask1,
        green0=green0,
        green1=green1,
        mask_diag0=diag0,
        mask_diag1=diag1,
        keypoints0=k0,
        keypoints1=k1,
        matches=matches,
        pts0=pts0,
        pts1=pts1,
        kp_size0=s0,
        kp_size1=s1,
        affine_M_1to0=M,
        affine_inliers=inl_aff,
        affine_rotation_deg=rot_deg,
        affine_tx=tx,
        affine_ty=ty,
        affine_scale=scale,
        affine_rmse=rmse,
        H_1to0=H,
        homography_inliers=inl_h,
        guided_mask=guided_mask,
        sfm_inliers=inl_pose,
        sfm_R=R,
        sfm_t=t,
        sfm_rot_deg=sfm_rot,
        sfm_yaw_deg=sfm_yaw,
        sfm_pitch_deg=sfm_pitch,
        sfm_roll_deg=sfm_roll,
        essential_inliers_raw=inl_e_raw,
        dropped_by_response=int(dropped_resp),
        dropped_by_size_ratio=int(dropped_size),
    )


In [ ]:
# Pair data preparation
pair_row = select_pair_row(pair_df, PAIR_ANCHOR_ID, PAIR_VAL_ID, PREVIEW_PAIR_INDEX)
anchor_id = int(pair_row['anchor_id'])
val_id = int(pair_row['val_id'])

r0 = train_df[train_df['id'].astype(int) == anchor_id].iloc[0]
r1 = train_df[train_df['id'].astype(int) == val_id].iloc[0]

img0_rgb, scale0 = load_train_image_cached(anchor_id, max_side=IMAGE_MAX_SIDE)
img1_rgb, scale1 = load_train_image_cached(val_id, max_side=IMAGE_MAX_SIDE)

K0 = scaled_K(r0, scale0)
K1 = scaled_K(r1, scale1)

gt0 = np.array([float(r0['x_pixel']) * scale0, float(r0['y_pixel']) * scale0], dtype=np.float64)
gt1 = np.array([float(r1['x_pixel']) * scale1, float(r1['y_pixel']) * scale1], dtype=np.float64)
gt_d = gt1 - gt0

print(f'pair: {anchor_id}->{val_id}')
print('img0:', img0_rgb.shape, 'scale0:', round(scale0, 4))
print('img1:', img1_rgb.shape, 'scale1:', round(scale1, 4))
print('GT delta (val-anchor):', np.round(gt_d, 2))

prev0 = preprocess_for_sift(img0_rgb, PREPROCESS_MODE)
prev1 = preprocess_for_sift(img1_rgb, PREPROCESS_MODE)
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(prev0, cmap='gray')
axes[0].set_title(f'Anchor preprocessed | id={anchor_id}')
axes[0].axis('off')
axes[1].imshow(prev1, cmap='gray')
axes[1].set_title(f'Validation preprocessed | id={val_id}')
axes[1].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Variant grid and run (Notebook-12 baseline + Notebook-14 ideas)
VARIANTS = [
    {
        'name': 'baseline_balanced',
        'preprocess_mode': 'gray_clahe',
        'mask_profile': 'none',
        'nfeatures': 9000,
        'contrast_thr': 0.03,
        'edge_thr': 12,
        'sigma': 1.6,
        'ratio_thr': 0.78,
        'mutual_check': False,
        'use_rootsift': False,
        'use_corner_guided': False,
        'use_grid_selection': False,
        'kp_response_min': 0.0,
        'size_ratio_min': 0.0,
        'size_ratio_max': 99.0,
        'essential_thr': ESSENTIAL_RANSAC_THR,
        'h_guide_err_px': H_GUIDE_ERR_PX,
        'mask_params': {},
    },
    {
        'name': 'rootsift_nongreen_mutual',
        'preprocess_mode': 'gray_clahe',
        'mask_profile': 'nongreen',
        'nfeatures': 9000,
        'contrast_thr': 0.04,
        'edge_thr': 12,
        'sigma': 1.6,
        'ratio_thr': 0.76,
        'mutual_check': True,
        'use_rootsift': True,
        'use_corner_guided': False,
        'use_grid_selection': True,
        'grid_shape': (8, 8),
        'grid_per_cell': 180,
        'kp_response_min': 0.010,
        'size_ratio_min': 0.45,
        'size_ratio_max': 2.40,
        'essential_thr': ESSENTIAL_RANSAC_THR,
        'h_guide_err_px': H_GUIDE_ERR_PX,
        'mask_params': {
            'green_h_low': 30,
            'green_h_high': 100,
            'green_s_min': 35,
            'green_v_min': 25,
        },
    },
    {
        'name': 'corner_guided_edge_mild',
        'preprocess_mode': 'gray_denoise_clahe',
        'mask_profile': 'edge_corner_mild',
        'nfeatures': 9500,
        'contrast_thr': 0.045,
        'edge_thr': 16,
        'sigma': 1.6,
        'ratio_thr': 0.75,
        'mutual_check': True,
        'use_rootsift': True,
        'use_corner_guided': True,
        'corner_max': 7000,
        'corner_quality': 0.01,
        'corner_min_dist': 6.0,
        'corner_block_size': 7,
        'corner_use_harris': False,
        'corner_kp_size': 16.0,
        'use_grid_selection': True,
        'grid_shape': (8, 8),
        'grid_per_cell': 180,
        'kp_response_min': 0.010,
        'size_ratio_min': 0.45,
        'size_ratio_max': 2.20,
        'essential_thr': ESSENTIAL_RANSAC_THR,
        'h_guide_err_px': H_GUIDE_ERR_PX,
        'mask_params': {
            'canny_low': 70,
            'canny_high': 170,
            'line_threshold': 60,
            'line_min_len': 55,
            'line_max_gap': 8,
            'corner_max': 1200,
            'corner_quality': 0.010,
            'corner_min_dist': 6,
            'min_coverage': 0.04,
        },
    },
    {
        'name': 'corner_guided_edge_strict',
        'preprocess_mode': 'gray_denoise_clahe',
        'mask_profile': 'edge_corner_strict',
        'nfeatures': 8000,
        'contrast_thr': 0.055,
        'edge_thr': 20,
        'sigma': 1.6,
        'ratio_thr': 0.72,
        'mutual_check': True,
        'use_rootsift': True,
        'use_corner_guided': True,
        'corner_max': 6000,
        'corner_quality': 0.012,
        'corner_min_dist': 7.0,
        'corner_block_size': 7,
        'corner_use_harris': False,
        'corner_kp_size': 16.0,
        'use_grid_selection': True,
        'grid_shape': (8, 8),
        'grid_per_cell': 160,
        'kp_response_min': 0.015,
        'size_ratio_min': 0.55,
        'size_ratio_max': 1.90,
        'essential_thr': ESSENTIAL_RANSAC_THR,
        'h_guide_err_px': H_GUIDE_ERR_PX,
        'mask_params': {
            'canny_low': 80,
            'canny_high': 190,
            'line_threshold': 72,
            'line_min_len': 68,
            'line_max_gap': 7,
            'corner_max': 900,
            'corner_quality': 0.015,
            'corner_min_dist': 7,
            'min_coverage': 0.03,
        },
    },
]

results: List[VariantResult] = []
for cfg in VARIANTS:
    out = run_sift_corner_variant(
        img0_rgb=img0_rgb,
        img1_rgb=img1_rgb,
        K0=K0,
        K1=K1,
        cfg=cfg,
    )
    results.append(out)

print('Finished variants:', [r.name for r in results])


In [ ]:
# Visualization helpers + 2-column panel
def to_rgb(gray: np.ndarray) -> np.ndarray:
    if gray.ndim == 2:
        return cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    return gray.copy()


def clip_pt(p: np.ndarray, w: int, h: int) -> Tuple[int, int]:
    x = int(np.clip(round(float(p[0])), 0, w - 1))
    y = int(np.clip(round(float(p[1])), 0, h - 1))
    return x, y


def draw_variant_pair(res: VariantResult, max_draw: int = 500) -> Tuple[np.ndarray, np.ndarray]:
    h0, w0 = res.gray0.shape[:2]

    left = _mask_tint(res.gray0, res.mask0)

    if res.affine_M_1to0 is not None:
        warped1 = cv2.warpAffine(res.gray1, res.affine_M_1to0, (w0, h0), flags=cv2.INTER_LINEAR, borderValue=0)
        p1w = affine_transform_points(res.affine_M_1to0, res.pts1)
    else:
        warped1 = cv2.resize(res.gray1, (w0, h0), interpolation=cv2.INTER_LINEAR)
        p1w = np.zeros_like(res.pts1)

    right = cv2.addWeighted(to_rgb(res.gray0), 0.45, to_rgb(warped1), 0.55, 0.0)

    n = len(res.matches)
    if n == 0:
        return left, right

    draw_idx = np.arange(n, dtype=np.int32)
    inl = res.affine_inliers if res.affine_inliers.shape[0] == n else np.zeros((n,), dtype=bool)

    if draw_idx.size > max_draw:
        inl_idx = np.where(inl)[0]
        out_idx = np.where(~inl)[0]
        if inl_idx.size >= max_draw:
            draw_idx = inl_idx[:max_draw]
        else:
            rem = max_draw - inl_idx.size
            draw_idx = np.concatenate([inl_idx, out_idx[:rem]])

    guided = res.guided_mask if res.guided_mask.shape[0] == n else np.zeros((n,), dtype=bool)

    for i in draw_idx:
        ok = bool(inl[i])
        gd = bool(guided[i])

        # green=inlier, orange=outlier+guided, red=outlier
        if ok:
            col = (0, 255, 0)
        elif gd:
            col = (255, 170, 0)
        else:
            col = (255, 0, 0)

        p0 = res.pts0[i]
        x0, y0 = clip_pt(p0, w0, h0)
        r0 = int(max(2, round(float(res.kp_size0[i]) * 0.35)))
        cv2.circle(left, (x0, y0), r0, col, 1, cv2.LINE_AA)
        cv2.circle(left, (x0, y0), 1, (255, 255, 0), -1, cv2.LINE_AA)

        if res.affine_M_1to0 is not None:
            p1 = p1w[i]
            x1, y1 = clip_pt(p1, w0, h0)
            r1 = int(max(2, round(float(res.kp_size1[i]) * 0.35)))
            cv2.circle(right, (x1, y1), r1, col, 1, cv2.LINE_AA)
            cv2.circle(right, (x1, y1), 1, (255, 255, 0), -1, cv2.LINE_AA)
            if ok:
                cv2.line(left, (x0, y0), (x1, y1), (0, 255, 255), 1, cv2.LINE_AA)

    return left, right


if len(results) == 0:
    raise RuntimeError('No variant results found. Run the variant cell first.')

fig, axes = plt.subplots(len(results), 2, figsize=(18, 5.8 * len(results)))
if len(results) == 1:
    axes = np.array([axes])

row_tags = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

for r, res in enumerate(results):
    left, right = draw_variant_pair(res, max_draw=MAX_DRAW_MATCHES)

    axes[r, 0].imshow(left)
    axes[r, 0].axis('off')
    axes[r, 1].imshow(right)
    axes[r, 1].axis('off')

    tag = row_tags[r] if r < len(row_tags) else f'({r + 1})'
    inl_a = int(res.affine_inliers.sum()) if res.affine_inliers.size else 0
    inl_h = int(res.homography_inliers.sum()) if res.homography_inliers.size else 0
    inl_s = int(res.sfm_inliers.sum()) if res.sfm_inliers.size else 0
    inl_er = int(res.essential_inliers_raw.sum()) if res.essential_inliers_raw.size else 0

    axes[r, 0].set_title(
        f"{res.name} {tag} | kp0={len(res.keypoints0)} kp1={len(res.keypoints1)} "
        f"matches={len(res.matches)} | affine inl={inl_a}"
    )
    axes[r, 1].set_title(
        f"rot={res.affine_rotation_deg:.2f}deg tx={res.affine_tx:.1f}px ty={res.affine_ty:.1f}px "
        f"scale={res.affine_scale:.4f} | H inl={inl_h} Eraw inl={inl_er} Eguided inl={inl_s}"
    )

plt.tight_layout()
plt.show()


In [ ]:
# Metrics table + best variant + planar diagnostic
def summarize_variant(res: VariantResult) -> Dict[str, object]:
    n = len(res.matches)
    inl_aff = int(res.affine_inliers.sum()) if res.affine_inliers.size else 0
    inl_h = int(res.homography_inliers.sum()) if res.homography_inliers.size else 0
    inl_e_raw = int(res.essential_inliers_raw.sum()) if res.essential_inliers_raw.size else 0
    inl_e = int(res.sfm_inliers.sum()) if res.sfm_inliers.size else 0

    rH = float(inl_h) / float(max(1, n))
    rEraw = float(inl_e_raw) / float(max(1, n))
    planar_dom = rH / max(1e-6, rEraw)

    gm0 = _pts_in_mask(res.green0, res.pts0)
    gm1 = _pts_in_mask(res.green1, res.pts1)
    mg0 = (100.0 * float(gm0.mean())) if n > 0 else np.nan
    mg1 = (100.0 * float(gm1.mean())) if n > 0 else np.nan

    sfm_tx, sfm_ty, sfm_tz = (np.nan, np.nan, np.nan)
    if res.sfm_t is not None:
        t = res.sfm_t.astype(float)
        tn = float(np.linalg.norm(t))
        if tn > 1e-12:
            t = t / tn
        sfm_tx, sfm_ty, sfm_tz = float(t[0]), float(t[1]), float(t[2])

    focus_score = float(inl_aff) - 0.10 * float((mg0 if pd.notna(mg0) else 100.0) + (mg1 if pd.notna(mg1) else 100.0))

    return {
        'variant': res.name,
        'preprocess': res.preprocess_mode,
        'mask_profile': res.mask_profile,
        'kp0': len(res.keypoints0),
        'kp1': len(res.keypoints1),
        'kp0_green_pct': _kp_green_fraction(res.keypoints0, res.green0),
        'kp1_green_pct': _kp_green_fraction(res.keypoints1, res.green1),
        'matches': n,
        'match_green0_pct': mg0,
        'match_green1_pct': mg1,
        'affine_inliers': inl_aff,
        'homography_inliers': inl_h,
        'essential_raw_inliers': inl_e_raw,
        'essential_guided_inliers': inl_e,
        'planar_dom_ratio_H_over_Eraw': planar_dom,
        'affine_rot_deg': res.affine_rotation_deg,
        'affine_tx_px': res.affine_tx,
        'affine_ty_px': res.affine_ty,
        'affine_scale': res.affine_scale,
        'affine_rmse_px': res.affine_rmse,
        'sfm_rot3d_deg': res.sfm_rot_deg,
        'sfm_yaw_deg': res.sfm_yaw_deg,
        'sfm_pitch_deg': res.sfm_pitch_deg,
        'sfm_roll_deg': res.sfm_roll_deg,
        'sfm_t_dir_x': sfm_tx,
        'sfm_t_dir_y': sfm_ty,
        'sfm_t_dir_z': sfm_tz,
        'mask_cov0': float(res.mask_diag0.get('mask_cov', np.nan)),
        'mask_cov1': float(res.mask_diag1.get('mask_cov', np.nan)),
        'mask_fallback0': int(res.mask_diag0.get('fallback', 0.0) > 0.5),
        'mask_fallback1': int(res.mask_diag1.get('fallback', 0.0) > 0.5),
        'drop_resp': int(res.dropped_by_response),
        'drop_size_ratio': int(res.dropped_by_size_ratio),
        'focus_score': focus_score,
    }


rows = [summarize_variant(r) for r in results]
metrics_df = pd.DataFrame(rows)
metrics_df = metrics_df.sort_values(
    ['focus_score', 'affine_inliers', 'matches'],
    ascending=[False, False, False],
).reset_index(drop=True)
display(metrics_df)

best = metrics_df.iloc[0]
print('Best variant:', best['variant'])
print(
    f"2D pose: rot={best['affine_rot_deg']:.2f}deg tx={best['affine_tx_px']:.2f}px "
    f"ty={best['affine_ty_px']:.2f}px scale={best['affine_scale']:.4f}"
)
print(
    f"Planar diagnostic H/Eraw={best['planar_dom_ratio_H_over_Eraw']:.2f} "
    f"(>1.3 often means homography-dominated pair)"
)
print(
    f"SfM orientation: yaw={best['sfm_yaw_deg']:.2f} pitch={best['sfm_pitch_deg']:.2f} "
    f"roll={best['sfm_roll_deg']:.2f} deg"
)
print(
    f"SfM t dir (unit): [{best['sfm_t_dir_x']:.3f}, {best['sfm_t_dir_y']:.3f}, {best['sfm_t_dir_z']:.3f}]"
)


In [ ]:
# Optional: raw match lines for one selected variant (notebook-1 style)
SELECT_VARIANT = 'corner_guided_edge_mild'
MAX_LINE_MATCHES = 180

cand = [r for r in results if r.name == SELECT_VARIANT]
if len(cand) == 0:
    sel_res = results[0]
    print(f'Variant {SELECT_VARIANT} not found. Using {sel_res.name}.')
else:
    sel_res = cand[0]

show_idx = np.arange(len(sel_res.matches), dtype=np.int32)
inl = sel_res.affine_inliers if sel_res.affine_inliers.shape[0] == len(sel_res.matches) else np.zeros((len(sel_res.matches),), bool)
if show_idx.size > MAX_LINE_MATCHES:
    inl_idx = np.where(inl)[0]
    out_idx = np.where(~inl)[0]
    need = int(MAX_LINE_MATCHES)
    if inl_idx.size >= need:
        show_idx = inl_idx[:need]
    else:
        show_idx = np.concatenate([inl_idx, out_idx[:need - inl_idx.size]])

vis0 = cv2.cvtColor(_ensure_gray_u8(sel_res.gray0), cv2.COLOR_GRAY2BGR)
vis1 = cv2.cvtColor(_ensure_gray_u8(sel_res.gray1), cv2.COLOR_GRAY2BGR)

draw_matches = [sel_res.matches[int(i)] for i in show_idx]
match_img = cv2.drawMatches(
    vis0,
    sel_res.keypoints0,
    vis1,
    sel_res.keypoints1,
    draw_matches,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)

plt.figure(figsize=(20, 8))
plt.imshow(cv2.cvtColor(match_img, cv2.COLOR_BGR2RGB))
plt.title(f"Raw matches | {sel_res.name} | drawn={len(draw_matches)}")
plt.axis('off')
plt.tight_layout()
plt.show()


## Hinweise
- Die Pipeline nutzt jetzt die robuste Pfad-/Dateilogik aus Notebook 12.
- Ideen aus Notebook 14 sind integriert (`RootSIFT`, `corner_guided`, `H-guided` Pose-Auswertung).
- Wenn du nur "12 + Fokus" willst, lasse `use_corner_guided=False` in allen Varianten.
- Falls zu wenige Matches entstehen: zuerst `mask_profile='none'` oder `'nongreen'`, dann `ratio_thr` leicht erhoehen (z. B. 0.78 -> 0.82).
